In [1]:
import numpy as np
import pyci
import psi4

In [2]:
# Lets intiate AOint() a subclass of psi4utils() 
aoint = pyci.utils.AOint('aug-cc-pv5z', 'He.xyz')
# saves all the necessary integrals as .npz files
aoint.save_all_aoints()
aoint.save_ao_dpints()
aoint.save_ao_qdints()
# run scf and get info for canonical HF orbitals
aoint.save_mo_info()
# lets load  ao_erints, mo_info and try to get the CIS matrix
ao_erints = np.load('ao_erints.npz')['electron_repulsion_aoints'] 
eps_a = np.load('mo_scf_info.npz')['eps_a']
Ca = np.load('mo_scf_info.npz')['Ca'] 
ao_dipoles = np.load('ao_dpints.npz')
ao_quadrupoles = np.load('ao_qdints.npz')
# lets convert our erints from AO basis to MO basis
mo_erints = aoint.eri_ao2mo(Ca, ao_erints)
mo_dpx = aoint.matrix_ao2mo(Ca, ao_dipoles['dpx_aoints'])
mo_dpy = aoint.matrix_ao2mo(Ca, ao_dipoles['dpy_aoints'])
mo_dpz = aoint.matrix_ao2mo(Ca, ao_dipoles['dpz_aoints'])
mo_qdxx = aoint.matrix_ao2mo(Ca, ao_quadrupoles['qdxx_aoints'])
mo_qdyy = aoint.matrix_ao2mo(Ca, ao_quadrupoles['qdyy_aoints'])
mo_qdzz = aoint.matrix_ao2mo(Ca, ao_quadrupoles['qdzz_aoints'])
del ao_erints, ao_dipoles, ao_quadrupoles

Size of the ERI tensor will be 0.33 Gb

Ground state SCF Energy : -2.86162693 

Nuclear repulsion energy : 0.00000000 

Total electronic energy : -2.86162693 



In [3]:
nbf, nmo, nso, na, nb, nocc, nvirt = aoint.get_orb_info(aoint.wfn)

 ORBITAL INFORMATION  
-----------------------
Basis functions   : 80  
Molecular orbitals: 80  
Spin Orbitals     : 160  
Alpha Orbitals    : 1  
Beta Orbitals     : 1  
Occupied Orbitals : 1  
Virtual Orbitals  : 79  



In [4]:
scf_energy = aoint.scf_energy
psi4_cisd = psi4.energy('CISD')

In [5]:
orbinfo = (nocc, nmo)
active_space = (nocc,nvirt)
singles = True
full_cis = True
doubles = True
doubles_iiaa = True
doubles_iiab = True
doubles_ijaa = True
doubles_ijab_A = True
doubles_ijab_B = True
options = [singles, full_cis, doubles,
           doubles_iiaa, doubles_iiab,doubles_ijaa,
           doubles_ijab_A, doubles_ijab_B]
csfs, num_csfs = pyci.configint.rcisd.generate_csfs(orbinfo, active_space, options)
num_csfs, sum(num_csfs)


([1, 79, 79, 3081, 0, 0, 0], 3240)

In [6]:
type(csfs)

list

In [7]:
HCISD = pyci.configint.rcisd.comp_hcisd(eps_a, mo_erints, scf_energy, orbinfo, active_space, options, ncore=2)

In [8]:
HCISD.shape

(3240, 3240)

In [9]:
vals, vecs = np.linalg.eigh(HCISD)

In [10]:
# Comparing pyci with psi4
print("pyci CISD E0: {e:16.16f}".format(e=vals[0]))
print("psi4 CISD E0: {e:16.16f}".format(e=psi4_cisd))
print("dE : {dE:1.2E}".format(dE=abs(psi4_cisd-vals[0])))


pyci CISD E0: -2.9032005295396814
psi4 CISD E0: -2.9032005295396721
dE : 9.33E-15


In [11]:
csf_dpx  = pyci.configint.rcisd.comp_oeprop_matrix(mo_dpx, orbinfo, active_space, options)
csf_dpy  = pyci.configint.rcisd.comp_oeprop_matrix(mo_dpy, orbinfo, active_space, options)
csf_dpz  = pyci.configint.rcisd.comp_oeprop_matrix(mo_dpz, orbinfo, active_space, options)
csf_qdxx  = pyci.configint.rcisd.comp_oeprop_matrix(mo_qdxx, orbinfo, active_space, options)
csf_qdyy  = pyci.configint.rcisd.comp_oeprop_matrix(mo_qdyy, orbinfo, active_space, options)
csf_qdzz  = pyci.configint.rcisd.comp_oeprop_matrix(mo_qdzz, orbinfo, active_space, options)

In [12]:
for i in range(10):
    x = np.dot(vecs[i], np.dot(csf_dpx,vecs[i]))
    y = np.dot(vecs[i], np.dot(csf_dpy,vecs[i]))
    z = np.dot(vecs[i], np.dot(csf_dpz,vecs[i]))
    print('{i:1d}, {x:3.16f}, {y:3.16f}, {z:3.16f}'.format(i=i, x=x,y=y,z=z))

0, -0.0067773767963527, 0.0108757353851584, -0.0004743206580641
1, 0.0039954628429837, -0.0062828208550420, 0.0011213957846038
2, 0.0222609603343498, -0.0082111775456521, -0.0291848959491724
3, -0.0468482270569540, -0.0080564316899645, -0.0454043745571115
4, 0.0448933152014449, 0.0085541372770124, -0.0323683741413576
5, 0.0800768660672546, -0.1204200291678243, 0.0054608622246160
6, -0.0185244584232795, -0.0345786517743020, 0.0297712174522053
7, -0.0053057037294914, -0.0278461039560670, -0.0322233529768708
8, -0.0544526634674205, 0.0162538211460427, -0.0182186962398267
9, 0.0270789539456493, -0.0117539173866759, 0.0272368086600925


In [ ]:
#  Dipoles data
# 0, 0.000283004784, 0.000035013676, 0.015705522296
# 1, 0.002917602220, 0.000673151423, 0.126488427923
# 2, 0.030531534767, 0.000594146873, -0.014433641987
# 3, 0.007507567538, 0.004000979317, 0.008836832031
# 4, 0.017980763541, 0.040301175514, 0.020415968090
# 5, 0.000220954675, 0.000120621738, 0.001989724419
# 6, 0.013474841181, -0.091742093587, -0.171431961356
# 7, 0.127483043223, 0.114134420569, -0.477323938172
# 8, -0.110211330355, -0.057932619221, -0.191399979168
# 9, -0.003589127667, -0.000760045647, -0.163052269270
# Quadrupoles data
# 0, -0.42547270, -0.43051704, -0.43176250,  1.287752
# 1, -5.02241121, -5.02126484, -5.02801822,  15.071694
# 2, -0.98754329, -2.71646871, -0.93718396,  4.641196
# 3, -1.39868692, -1.12545165, -2.82703797,  5.351177
# 4, -2.96151585, -1.38259467, -1.53044711,  5.874558
# 5, -0.76163766, -0.69778652, -1.07033477,  2.529759
# 6, -5.49423508, -5.40268524, -7.17795632,  18.074877
# 7, -2.23722944, -3.72175897, -3.77558908,  9.734577
# 8, -3.58304603, -2.22482522, -3.66498503,  9.472856
# 9, -0.69069120, -0.72179435, -0.86955816,  2.282044

In [ ]:
for i in range(10):
    xx = np.dot(vecs[i], np.dot(csf_qdxx,vecs[i]))
    yy = np.dot(vecs[i], np.dot(csf_qdyy,vecs[i]))
    zz = np.dot(vecs[i], np.dot(csf_qdzz,vecs[i]))
    r2 = abs(xx)+abs(yy)+abs(zz)
    print('{i:1d}, {xx:3.8f}, {yy:3.8f}, {zz:3.8f}, {r2: 3.6f}'.format(i=i, xx=xx,yy=yy,zz=zz, r2=r2))